# CARLA Project - Car and distance detection

### Package installation

Install the required packages with the following command (run it where the `requirements.txt` file is located):

```bash
pip install -r requirements.txt
```

In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
import torch
from torchvision import transforms, models
from PIL import Image
from queue import Queue
from queue import Empty
from ultralytics import YOLO

pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\seren\.conda\envs\carla\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)

# client.load_world('Town01')
world = client.get_world()
spectator = world.get_spectator()
# world.set_weather(carla.WeatherParameters.ClearNoon)

In [3]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-10, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.5, z=1.5), carla.Rotation(pitch=-10)), width=800, height=600, frequency = 0):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera_bp.set_attribute('sensor_tick', str(frequency))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x=1.5, z=1.5), carla.Rotation(pitch=-10)), horizontal_fov = 35, vertical_fov = 20):
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar = world.spawn_actor(radar_bp, transform, attach_to=attach_to)
    return radar

In [4]:
for actor in world.get_actors().filter('*vehicle*'):
  actor.destroy()

for actor in world.get_actors().filter('*sensor*'):
  actor.destroy()

In [14]:
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

sensor_transform = carla.Transform(carla.Location(x=-0.16, y=-0.9, z=2.4), carla.Rotation(yaw=-100))

vehicle = spawn_vehicle(spawn_index=1, pattern="vehicle.dodge.charger_2020")
camera = spawn_camera(attach_to=vehicle, transform=sensor_transform, width=320, height=320)
radar = spawn_radar(attach_to=vehicle, transform=sensor_transform, horizontal_fov=60, vertical_fov=40)
velocity_range = 7.5 # m/s

video_output = np.zeros((320, 320, 4), dtype=np.uint8)

image_queue = Queue()
radar_queue = Queue()

def sensor_callback(data, queue):
    queue.put(data)

camera.listen(lambda data: sensor_callback(data, image_queue))
radar.listen(lambda data: sensor_callback(data, radar_queue))

vehicle.set_autopilot(True)

cv2.namedWindow('Left Side Camera', cv2.WINDOW_AUTOSIZE)

model = YOLO("runs/detect/train2/weights/best.pt")

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        
        world.tick()

        try:
            # Get the data once it's received.
            image_data = image_queue.get(True, 1.0)
            radar_data = radar_queue.get(True, 1.0)
        except Empty:
            print("[Warning] Some sensor data has been missed")
            continue

        # sync the data
        assert image_data.frame == radar_data.frame

        # Retrive the image data
        video_output = np.reshape(np.copy(image_data.raw_data), (image_data.height, image_data.width, 4))

        # ----------- RADAR data -----------

        points = np.frombuffer(radar_data.raw_data, dtype=np.dtype('f4'))
        points = np.reshape(points, (len(radar_data), 4))

        # Filter out points under the radar (altitude < 0) to avoid road surface and other objects
        points = points[points[:, 2] <= 0] 

        # [vel, azimuth, altitude, depth]
        velocities = points[:, 0]
        azimuth = points[:, 1]
        altitude = points[:, 2]
        depth = points[:, 3]     

        # Convert spherical coordinates to Cartesian (sensor coordinate system)
        X = depth * np.cos(altitude) * np.sin(azimuth)
        Y = depth * np.sin(altitude)
        Z = depth * np.cos(altitude) * np.cos(azimuth)

        # Stack into a single array (shape: Nx3)
        cartesian_points = np.stack((X, Y, Z), axis=1)

        # Build the K projection matrix:
        # K = [[Fx,  0, image_w/2],
        #      [ 0, Fy, image_h/2],
        #      [ 0,  0,         1]]
        image_w = 320 #image_size_x
        image_h = 320 #image_size_y

        #since we are using the standard camera fov
        camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
        fov = camera_bp.get_attribute("fov").as_float() 
        focal = image_w / (2.0 * np.tan(fov * np.pi / 360.0))

        # In this case Fx and Fy are the same since the pixel aspect ratio is 1
        K = np.identity(3)
        K[0, 0] = K[1, 1] = focal
        K[0, 2] = image_w / 2.0
        K[1, 2] = image_h / 2.0

        # cartesian_points.T is a 3xN matrix - can be multiplied directly by K
        points_2d = np.dot(K, cartesian_points.T)

        # Normalize the points
        points_2d = np.array([
            points_2d[0, :] / points_2d[2, :],
            points_2d[1, :] / points_2d[2, :],
            points_2d[2, :]])
        
        points_2d = points_2d.T

        frame = video_output.copy()  # Copy to avoid modifying the original


        # ----------- YOLO evaluation -----------

        eval_img = video_output[:, :, :3]
        results = model(eval_img, imgsz=320)

        for result in results:
            for i in range(len(result)):
                min_distance = 1000
                confidence = result.boxes.conf[i]
                if confidence < 0.5: # skip if confidence is low
                    continue
                box = result.boxes.xyxy[i]
                for j in range(len(points_2d)):
                    if box[0] < points_2d[i][0] < box[2] and box[1] < points_2d[i][1] < box[3]:
                        if depth[i] < min_distance:
                            min_distance = depth[i]
                if min_distance < 1000:
                    print("Car detected at distance: ", min_distance)
            # for box in result.boxes.xyxy:
            #     # look for the closest point to the ego vehicle
            #     min_distance = 1000
            #     for i in range(len(points_2d)):
            #         if box[0] < points_2d[i][0] < box[2] and box[1] < points_2d[i][1] < box[3]:
            #             if depth[i] < min_distance:
            #                 min_distance = depth[i]
            #     if min_distance < 1000:
            #         print("Car detected at distance: ", min_distance)


        # ----------- DISPLAY ON SCREEN FOR DEBUG -----------

        # Loop through all projected 2D points
        for point in points_2d:
            x, y = int(point[0]), int(point[1])  # Convert to integer pixel coordinates

            # Draw the point only if it is inside the image frame
            if 0 <= x < frame.shape[1] and 0 <= y < frame.shape[0]:
                cv2.circle(frame, (x, y), radius=2, color=(0, 0, 255), thickness=-1)  # Red dot

        
        for result in results:
            # print(result.boxes.xyxy)
            for box in result.boxes.xyxy:
                cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 1)

        # Show the image with radar points overlaid
        cv2.imshow('Left Side Camera', frame)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()
    radar.destroy()
    settings = world.get_settings()
    settings.synchronous_mode = False
    settings.no_rendering_mode = False
    settings.fixed_delta_seconds = None
    world.apply_settings(settings)

Ultralytics YOLOv8.0.20  Python-3.7.16 torch-1.10.0+cu113 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
